# Mount drive and append path to PYTONPATH


In [ ]:
import os
import sys

from google.colab import drive, files, runtime

drive.mount("/content/drive")
sys.path.append("/content/drive/MyDrive/DeepLCMS/train_google_colab")

# Import and install libraries

In [ ]:
%%capture
!pip install lightning
!pip install timm
!pip install torchinfo
!pip install scikit-posthocs
!pip install optuna
!pip install torchcam

In [ ]:
import colab_functions
import colab_utils
import pandas as pd
import prepare_data
import timm
import train_NN
from lightning.pytorch import loggers, callbacks, tuner, trainer, LightningModule

import optuna
import torchmetrics
import timm
import torch

from pathlib import Path
from tqdm import tqdm
import pickle

In [ ]:
# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Unzip data

In [ ]:
!unzip -q "/content/drive/MyDrive/DeepLCMS/ST001618_Opium_study_LC_MS_500.zip"

# Check if GPU is used

In [ ]:
device = colab_functions.get_device()

# Getting a tunable model

In [ ]:
class TunedPretrainedModel(LightningModule):
    def __init__(
        self,
        pretrained_model_name,
        hyperparameters,
        learning_rate,
        freeze=True,
    ):
        super().__init__()
        self.hyperparameters = hyperparameters
        self.pretrained_model_name = pretrained_model_name
        self.model = timm.create_model(
            pretrained_model_name, pretrained=True, num_classes=1
        )
        self.learning_rate = learning_rate
        self.loss_fn = torch.nn.BCEWithLogitsLoss()
        self.accuracy = torchmetrics.classification.BinaryAccuracy()
        self.f1 = torchmetrics.classification.BinaryF1Score()
        self.precision = torchmetrics.classification.BinaryPrecision()
        self.recall = torchmetrics.classification.BinaryRecall()

        if freeze:
            # Freeze all layers
            for param in self.model.parameters():
                param.requires_grad = False

            # Get the last layer
            last_layer = None
            for child in self.model.named_children():
                last_layer = child

            # Unfreeze the last layer
            if last_layer is not None:
                for param in last_layer[1].parameters():
                    param.requires_grad = True

    def forward(self, x: torch.Tensor):
        x = self.model(x)
        return x

    def common_step(self, batch, batch_idx):
        x, y = batch
        y_pred_logits = self(x).squeeze()
        loss = self.loss_fn(y_pred_logits, y.float())
        return loss, y_pred_logits, y

    def log_metrics(
        self,
        prefix,
        loss,
        accuracy,
        f1,
        precision,
        recall,
    ):
        self.log_dict(
            {
                f"{prefix}_loss": loss,
                f"{prefix}_accuracy": accuracy,
                f"{prefix}_f1": f1,
                f"{prefix}_precision": precision,
                f"{prefix}_recall": recall,
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )

    def training_step(self, batch, batch_idx):
        loss, y_pred_logits, y = self.common_step(batch, batch_idx)
        accuracy = self.accuracy(y_pred_logits, y)
        f1 = self.f1(y_pred_logits, y)
        precision = self.precision(y_pred_logits, y)
        recall = self.recall(y_pred_logits, y)

        self.log_metrics("train", loss, accuracy, f1, precision, recall)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, y_pred_logits, y = self.common_step(batch, batch_idx)
        accuracy = self.accuracy(y_pred_logits, y)
        f1 = self.f1(y_pred_logits, y)
        precision = self.precision(y_pred_logits, y)
        recall = self.recall(y_pred_logits, y)

        self.log_metrics("val", loss, accuracy, f1, precision, recall)
        return loss

    def predict_step(self, batch, batch_idx, dataloader_idx):
        if isinstance(batch, list):
            input_tensor = batch[0]
            return self(input_tensor)
        else:
            print("Input Shape:", batch.shape)
            return self(batch)

    def configure_optimizers(self):
        optimizer = None

        if self.hyperparameters["optimizer"] == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=0.001, weight_decay=2e-5)
        elif self.hyperparameters["optimizer"] == "AdamW":
            optimizer = torch.optim.AdamW(
                self.parameters(), lr=0.001, weight_decay=2e-5
            )
        elif self.hyperparameters["optimizer"] == "Adamax":
            optimizer = torch.optim.AdamW(
                self.parameters(), lr=0.001, weight_decay=2e-5
            )
        elif self.hyperparameters["optimizer"] == "RMSprop":
            optimizer = torch.optim.RMSprop(
                self.parameters(), lr=0.001, weight_decay=2e-5
            )
        else:
            raise ValueError(
                f"Unsupported optimizer: {self.hyperparameters['optimizer']}"
            )

        scheduler = None

        if self.hyperparameters["scheduler"] == "ReduceLROnPlateau":
            scheduler = {
                "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(
                    optimizer, mode="min", factor=0.1, patience=3
                ),
                "interval": "epoch",
                "monitor": "val_loss",
            }
        elif self.hyperparameters["scheduler"] == "CosineAnnealingLR":
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer, T_max=50, eta_min=0
            )

        return [optimizer], [scheduler]


def objective(trial):
    hyperparameters = {
        "optimizer": trial.suggest_categorical(
            "optimizer", ["Adam", "AdamW", "Adamax", "RMSprop"]
        ),
        "scheduler": trial.suggest_categorical(
            "scheduler", ["ReduceLROnPlateau", "CosineAnnealingLR"]
        ),
    }

    model = TunedPretrainedModel(
        hyperparameters=hyperparameters,
        pretrained_model_name="convnext_large_mlp.clip_laion2b_augreg_ft_in1k_384",
        learning_rate=0.001,
    )
    logger = loggers.CSVLogger("logs", name=str(trial.number))
    trainer_ = trainer.Trainer(
        logger=logger,
        log_every_n_steps=1,
        max_epochs=50,
        callbacks=[
            callbacks.EarlyStopping(monitor="val_loss", patience=3),
            optuna.integration.PyTorchLightningPruningCallback(
                trial, monitor="val_loss"
            ),
        ],
    )

    trainer_.fit(model=model, datamodule=datamodule)

    return trainer_.callback_metrics["val_loss"].item()

In [ ]:
PRETRAINED_MODEL = "convnext_large_mlp.clip_laion2b_augreg_ft_in1k_384"

model = train_NN.PretrainedModel(
    pretrained_model_name=PRETRAINED_MODEL, learning_rate=0.001
)
datamodule = prepare_data.LCMSDataModule(
    model,
    data_dir=Path("/content/ST001618_Opium_study_LC_MS_500"),
)
model.show_architecture()

In [ ]:
# the total number of possible combinations is 15
# based on this : nCr = n! / (r! * (n - r)!)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

In [ ]:
with open("optuna_params.pickle", "wb") as handle:
    pickle.dump(study.best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open("optuna_params.pickle", "rb") as handle:
    optuna_params = pickle.load(handle)

print(study.best_params == optuna_params)

In [ ]:
# save the result to Google drive
results_df = colab_functions.get_experiment_results()
results_df.to_csv("pretrained_model_results.csv", index=False)

!cp -r "/content/pretrained_model_results.csv" "/content/drive/MyDrive/train_google_colab"
!cp -r "/content/optuna_params.pickle" "/content/drive/MyDrive/train_google_colab"
# Close runtime

runtime.unassign()

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_contour(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

# Evaluate results

In [ ]:
results_df = colab_functions.get_experiment_results().assign(
    experiment=lambda df: df.experiment.astype(int)
)
optuna_trials = pd.read_csv("optuna_trials.csv")

# merge results_df with optuna_trials so that we have access to the full training
# data with all epochs
# this is needed since optuna made a decision based on overfitted data

df = results_df.merge(optuna_trials, left_on="experiment", right_on="number")

Optuna originally selected trial #27, which only achieved a validation loss of 0.1353. This is because Optuna considers the validation loss of the last epoch before terminating the trial due to overfitting. Therefore, the final conclusion reached by Optuna is based on an already overfitted model. Based on the learning curves logged, we can determine the best conditions and the number of epochs we should train our model for.

These are the validation losses regarding trial #27:

In [ ]:
(df.query("experiment == 27 and variable.str.contains('val_loss')")).sort_values(
    by="epoch"
)

The absolute minimum validation loss reached during the optimization procedure was 0.1289 at epoch 3 for experiment 12. However, by the next epoch, the model was overfitted and recorded a final validation loss of 0.2212, which is higher than the validation loss of trial #27. Therefore, trial #27 was selected over experiment 12.

In [ ]:
(
    df.loc[lambda df: df.groupby("variable")["value_x"].idxmin(), :].query(
        "variable.str.contains('val_loss') "
    )
)

These experiments represent the overall best trials and corresponding epochs per metric monitored. As you can see, trial #12 is still present with a validation accuracy of 0.98. Most of these trials used the `Adam` optimizer and `CosineAnnealingLR` scheduler. Based on these findings, we can opt for the conditions described for trial #12.

In [ ]:
(
    df.loc[lambda df: df.groupby("variable")["value_x"].idxmax(), :].query(
        "~variable.str.contains('train|loss')"
    )
)